# IBM Applied Data Science Capstone Course by Coursera  
## Week 3 Part 1

## 1.Import libraries

In [2]:
conda install -c conda-forge geopy

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\asus\Anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |   py37hc8dfbb8_1         149 KB  conda-forge
    conda-4.8.3                |   py37hc8dfbb8_1         3.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    openssl-1.1.1e             |       hfa6e2cd_0         4.7 MB  conda-forge
    python_abi-3.7             |          1_cp37m           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         8.0 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-p

In [1]:
conda install -c anaconda beautifulsoup4

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\asus\Anaconda3

  added / updated specs:
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.8.2       |           py37_0         161 KB  anaconda
    ca-certificates-2020.1.1   |                0         165 KB  anaconda
    certifi-2019.11.28         |           py37_0         157 KB  anaconda
    conda-4.8.3                |           py37_0         3.0 MB  anaconda
    openssl-1.1.1              |       he774522_0         5.7 MB  anaconda
    ------------------------------------------------------------
                                           Total:         9.3 MB

The following packages will be UPDATED:

  beautifulsoup4     pkgs/main::beautifulsoup4-4.7.1-py37_1 --> anaconda::beautifulsoup4-4.8.2-py37_0
  ca-certificates    conda-forge::ca-certificates

In [1]:
conda install -c conda-forge/label/cf201901 folium

Solving environment: ...working... 
  - anaconda/win-64::conda-4.8.3-py37_0
  - defaults/win-64::conda-4.8.3-py37_0done

## Package Plan ##

  environment location: C:\Users\asus\Anaconda3

  added / updated specs:
    - folium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2018.11.29 |       ha4d7672_0         179 KB  conda-forge/label/cf201901
    certifi-2018.11.29         |        py37_1000         144 KB  conda-forge/label/cf201901
    folium-0.7.0               |             py_0          54 KB  conda-forge/label/cf201901
    ------------------------------------------------------------
                                           Total:         377 KB

The following packages will be UPDATED:

  folium                     conda-forge::folium-0.5.0-py_0 --> conda-forge/label/cf201901::folium-0.7.0-py_0

The following packages will be SUPERSEDED by a higher-priority c

In [23]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stagz
from sklearn.cluster import KMeans

import folium # map rendering library
import csv

print("Libraries imported.")

Libraries imported.


## 2. Scrap data from wikipedia page

In [39]:
url = requests.get('https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050.').text
soup = BeautifulSoup(url,'lxml')

## Locate Table and use tags to find postal code by Borough and Neighbourhood

In [40]:
table_post = soup.find('table')
fields = table_post.find_all('td')

In [41]:
postcode = []
borough = []
neighbourhood = []

In [42]:
for i in range(0, len(fields), 3):
    postcode.append(fields[i].text.strip())
    borough.append(fields[i+1].text.strip())
    neighbourhood.append(fields[i+2].text.strip())

In [43]:
df_pc = pd.DataFrame(data=[postcode, borough, neighbourhood]).transpose()
df_pc.columns = ['Postcode', 'Borough', 'Neighbourhood']
df_pc.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## 3.Drop cells with a borough that is "Not assigned"

In [58]:
# drop cells with a borough that is Not assigned
toronto_df_dropna = df_pc[df_pc.Borough != "Not assigned"].reset_index(drop=True)
toronto_df_dropna.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


## 4.Group neighborhoods in the same borough

In [61]:
# group neighborhoods in the same borough
toronto_df_grouped = toronto_df_dropna.groupby(["Postcode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_df_grouped.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## 5. For Neighborhood="Not assigned", make the value the same as Borough

In [63]:
# for Neighborhood="Not assigned", make the value the same as Borough
for index, row in toronto_df_grouped.iterrows():
    if row["Neighbourhood"] == "Not assigned":
        row["Neighbourhood"] = row["Borough"]
        
toronto_df_grouped.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## 6. Check whether it is the same as required by the question

In [66]:
# create a new test dataframe
column_names = ["Postcode", "Borough", "Neighbourhood"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(toronto_df_grouped[toronto_df_grouped["Postcode"]==postcode], ignore_index=True)
    
test_df

,Postcode,Borough,Neighbourhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,"Woodbine Gardens, Parkview Hill"
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,"Maryvale, Wexford"
7,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown"
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara"


## 7. Finally, print the number of rows of the cleaned dataframe

In [67]:
# print the number of rows of the cleaned dataframe
toronto_df_grouped.shape

(103, 3)